In [1]:
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import relativedelta
pd.set_option('display.max_columns', None)
import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import linregress
import seaborn as sns
sns.set(context='notebook')
sns.set_style("whitegrid", {'axes.grid' : False})
plt.tight_layout()
from pandas.tseries.offsets import MonthEnd

import warnings
warnings.filterwarnings("ignore")

loan_data_temp = pd.read_csv('C:\\Users\\ryanc\\Downloads\\loan_data_2007_to_2018.csv\\accepted_2007_to_2018Q4.csv',usecols=['id','loan_amnt','int_rate','installment','issue_d','last_pymnt_d','term'])

<Figure size 640x480 with 0 Axes>

In [5]:
loan_data_temp = loan_data_temp[loan_data_temp['issue_d'].notnull()]
loan_data_temp['issue_dt'] = pd.to_datetime(loan_data_temp['issue_d']) + pd.offsets.MonthEnd(0)

#事实表
LoanDf = pd.DataFrame()

for dt in pd.date_range(start='2015-12-31', end='2018-12-31', freq='ME'):
    obs_dt = dt
    mth = pd.to_datetime(obs_dt).strftime('%Y%m')
    
    loan_data = loan_data_temp.loc[(loan_data_temp['issue_dt']>'2015-11-30')&(loan_data_temp['issue_dt']<=obs_dt)]
    
    loan_data['issue_dt'] = pd.to_datetime(loan_data['issue_dt'])
    
    loan_data['obs_dt'] = obs_dt
    loan_data['obs_dt'] = pd.to_datetime(loan_data['obs_dt'])
    
    loan_data['last_pymt_dt'] = pd.to_datetime(loan_data['last_pymnt_d']) + pd.offsets.MonthEnd(0)
    
    loan_data['term'] = loan_data['term'].astype(str).str.replace(' months','')
    loan_data['term'] = loan_data['term'].astype(int)
    
    loan_data['mob'] = loan_data['obs_dt'].dt.to_period('M').astype('int64') - \
                       loan_data['issue_dt'].dt.to_period('M').astype('int64')

    loan_data = loan_data[~(loan_data['mob']>loan_data['term'])]
    
    loan_data['ost_bal'] = loan_data['installment'] * loan_data['mob']

    loan_data = loan_data[~(loan_data['ost_bal']>loan_data['loan_amnt'])]
    
    loan_data['mths_odue'] = loan_data['obs_dt'].dt.to_period('M').astype('int64') - \
                             loan_data['last_pymt_dt'].dt.to_period('M').astype('int64')
    
    loan_data['mths_odue'] = np.where(loan_data['mths_odue']>0,loan_data['mths_odue'],0)

    loan_data['mths_odue'] = np.where(loan_data['mths_odue']==loan_data['term'],loan_data['term'],loan_data['mths_odue'])
        
    loan_data['ost_bal'] = (loan_data['installment'] * loan_data['term']) - (loan_data['installment'] * loan_data['mob']) + (loan_data['installment'] * loan_data['mths_odue'])
    
    loan_data['odue_amt'] = loan_data['installment'] * loan_data['mths_odue']

    print('# Aggregating loan data as of',dt,'is completed. Rows ct:',len(loan_data))
    LoanDf = pd.concat([LoanDf,loan_data])
    print('# Concat loan data as of',dt,'is completed. Rows ct:',len(LoanDf))

#LoanDf.to_csv('loan_event_2016_2018',index=False)

# Aggregating loan data as of 2015-12-31 00:00:00 is completed. Rows ct: 44343
# Concat loan data as of 2015-12-31 00:00:00 is completed. Rows ct: 44343
# Aggregating loan data as of 2016-01-31 00:00:00 is completed. Rows ct: 76709
# Concat loan data as of 2016-01-31 00:00:00 is completed. Rows ct: 121052
# Aggregating loan data as of 2016-02-29 00:00:00 is completed. Rows ct: 116238
# Concat loan data as of 2016-02-29 00:00:00 is completed. Rows ct: 237290
# Aggregating loan data as of 2016-03-31 00:00:00 is completed. Rows ct: 178230
# Concat loan data as of 2016-03-31 00:00:00 is completed. Rows ct: 415520
# Aggregating loan data as of 2016-04-30 00:00:00 is completed. Rows ct: 214662
# Concat loan data as of 2016-04-30 00:00:00 is completed. Rows ct: 630182
# Aggregating loan data as of 2016-05-31 00:00:00 is completed. Rows ct: 243065
# Concat loan data as of 2016-05-31 00:00:00 is completed. Rows ct: 873247
# Aggregating loan data as of 2016-06-30 00:00:00 is completed. Rows ct: 

In [215]:
data_l = LoanDf[LoanDf['obs_dt']=='2015-12-31']
data_t = LoanDf[LoanDf['obs_dt']=='2016-01-31']
data_n = LoanDf[LoanDf['obs_dt']=='2016-02-29']


In [217]:

data_l = data_l.rename(columns={'odue_amt':'odue_amt_l','obs_dt':'obs_dt_l','mths_odue':'mths_odue_l'})
data_l = data_l[['id','obs_dt_l','odue_amt_l','mths_odue_l']]

data_t = data_t.rename(columns={'odue_amt':'odue_amt_t','obs_dt':'obs_dt_t','mths_odue':'mths_odue_t'})
data_t = data_t[['id','obs_dt_t','odue_amt_t','mths_odue_t']]

data_n = data_n.rename(columns={'odue_amt':'odue_amt_n','obs_dt':'obs_dt_n','mths_odue':'mths_odue_n'})
data_n = data_n[['id','obs_dt_n','odue_amt_n','mths_odue_n']]

data = data_l.merge(data_t,on='id',how='left').merge(data_n,on='id',how='left')

#data.to_csv('201611.csv',index=False)

In [ ]:
LoanDf = LoanDf.reset_index()

LoanDf.loc[LoanDf['mths_odue']==0,'curr_bal'] = LoanDf['installment']
LoanDf.loc[LoanDf['mths_odue']>=1,'M1_bal'] = LoanDf['installment']

M2 = LoanDf.loc[LoanDf['mths_odue']==1]
M2 = M2.rename(columns={'M1_bal':'M2_bal'})
M2['obs_dt_m2'] = M2['obs_dt'] + MonthEnd(1)
M2 = M2[['id','obs_dt_m2','M2_bal']]

LoanDf = LoanDf.merge(M2,left_on=['id','obs_dt'],right_on = ['id','obs_dt_m2'],how='left')
LoanDf.loc[LoanDf['mths_odue']>=1,'M1_bal'] = LoanDf['installment']

M3 = LoanDf.loc[LoanDf['mths_odue']==2]
M3 = M3.rename(columns={'M2_bal':'M3_bal'})
M3['obs_dt_m3'] = M3['obs_dt'] + MonthEnd(1)
M3 = M3[['id','obs_dt_m3','M3_bal']]

LoanDf = LoanDf.merge(M3,left_on=['id','obs_dt'],right_on = ['id','obs_dt_m3'],how='left')

LoanDf.loc[LoanDf['mths_odue']>=2,'M2_bal'] = LoanDf['installment']

M4 = LoanDf.loc[LoanDf['mths_odue']==3]
M4 = M4.rename(columns={'M3_bal':'M4_bal'})
M4['obs_dt_m4'] = M4['obs_dt'] + MonthEnd(1)
M4 = M4[['id','obs_dt_m4','M4_bal']]

LoanDf = LoanDf.merge(M4,left_on=['id','obs_dt'],right_on = ['id','obs_dt_m4'],how='left')

LoanDf.loc[LoanDf['mths_odue']>=3,'M3_bal'] = LoanDf['installment']

M5 = LoanDf.loc[LoanDf['mths_odue']==4]
M5 = M5.rename(columns={'M4_bal':'M5_bal'})
M5['obs_dt_m5'] = M5['obs_dt'] + MonthEnd(1)
M5 = M5[['id','obs_dt_m5','M5_bal']]

LoanDf = LoanDf.merge(M5,left_on=['id','obs_dt'],right_on = ['id','obs_dt_m5'],how='left')

LoanDf.loc[LoanDf['mths_odue']>=4,'M4_bal'] = LoanDf['installment']

M6 = LoanDf.loc[LoanDf['mths_odue']==5]
M6 = M6.rename(columns={'M5_bal':'M6_bal'})
M6['obs_dt_m6'] = M6['obs_dt'] + MonthEnd(1)
M6 = M6[['id','obs_dt_m6','M6_bal']]

LoanDf = LoanDf.merge(M6,left_on=['id','obs_dt'],right_on = ['id','obs_dt_m6'],how='left')

LoanDf.loc[LoanDf['mths_odue']>=5,'M5_bal'] = LoanDf['installment']

M7 = LoanDf.loc[LoanDf['mths_odue']==6]
M7 = M7.rename(columns={'M6_bal':'M7_bal'})
M7['obs_dt_m7'] = M7['obs_dt'] + MonthEnd(1)
M7 = M7[['id','obs_dt_m7','M7_bal']]

LoanDf = LoanDf.merge(M7,left_on=['id','obs_dt'],right_on = ['id','obs_dt_m7'],how='left')

LoanDf.loc[LoanDf['mths_odue']>=6,'M6_bal'] = LoanDf['installment']

In [318]:
migr_1 = LoanDf[['obs_dt','curr_bal','M1_bal','M2_bal','M3_bal','M4_bal','M5_bal','M6_bal','M7_bal']].groupby(['obs_dt'],as_index=False).sum()
migr_1['obs_dt'] = pd.to_datetime(migr_1['obs_dt']).dt.strftime('%Y%m')

migr_1 = migr_1.rename(columns={'obs_dt':'month','M1_bal':'M1','M2_bal':'M2','M3_bal':'M3','M4_bal':'M4','M5_bal':'M5','M6_bal':'M6','M7_bal':'M7'})

migr_1 = migr_1.set_index(migr_1['month'])
migr_1 = migr_1.drop(columns='month')
migr_1 = migr_1.T

In [324]:
migr_df = migr_1/migr_1.shift(1,axis=1).shift(1,axis=0)
migr_df = migr_df.iloc[1:,24:37]
migr_df.index = ['M0->M1','M1->M2','M2->M3','M3->M4','M4->M5','M5->M6','M6->M7']
migr_df = migr_df.iloc[:-1,:-1]
#migr_df = migr_df.round(2).astype(str) + '%'

In [326]:
migr_df

month,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811
M0->M1,0.286645,0.300666,0.317755,0.334015,0.351540,0.362291,0.367048,0.372669,0.375345,0.376940,0.376388,0.375428
M1->M2,0.999998,0.999773,0.999602,0.998681,0.993905,0.987784,0.974906,0.965553,0.955645,0.948115,0.945818,0.949410
M2->M3,0.999998,0.999753,0.999580,0.998599,0.993604,0.987211,0.973981,0.964162,0.953893,0.946210,0.943743,0.947766
M3->M4,0.999998,0.999735,0.999550,0.998490,0.993216,0.986724,0.972590,0.962661,0.952133,0.944139,0.941608,0.945608
M4->M5,0.999998,0.999713,0.999514,0.998355,0.992861,0.986039,0.971408,0.960878,0.950220,0.941887,0.939417,0.943533
M5->M6,0.999997,0.999698,0.999497,0.998254,0.992448,0.985483,0.969971,0.959390,0.947834,0.939813,0.936863,0.941215
